<img src='https://www.wekeo.eu/images/logo-large.png' div style="text-align:center" width='25%'> 


<div class="alert alert-block alert-success">
<h3>CMEMS Reanalysis: Daily and Monthly Mean Time Series, Annual Trends, Mean Seasonals, and Seasonal Anomalies of the Sea Surface Temperature in the Adriatic Sea</h3></div>

**Author and Team Leader:**  Özlem ÖZALP <br>
**Co-authors:**  Antonio VECOLI and Jacopo ALESSANDRI <br>
**Copyright:** 2022 <br>
**License:** MIT


The Sea Surface Temperature (SST) is an important physical characteristic of the oceans and is the one of the vital component of the <a href="https://gcos.wmo.int/en/essential-climate-variables/sst" target="_blank">climate system</a> . The Adriatic Sea is an elongated basin, located in the central Mediterranean, between the Italian peninsula and the Balkans.The CMEMS Mediterranean Sea Physics Reanalysis time series is provided since 01/01/1987. The SST is defined by selecting the first vertical level of the daily mean of Potential Temperature within the variable name “thetao”. The data is available and can be downloded from the following <a href="https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1" target="_blank">link</a>

# **Introduction to WEkEO Notebook**

### **Data used**

| Product Description | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:-----------------:|
| CMEMS Reanalysis: Daily Mean Potential temperature from 1987 to 2019 | MEDSEA_MULTIYEAR_PHY_006_004  | <a href="https://resources.marine.copernicus.eu/product-detail/MEDSEA_MULTIYEAR_PHY_006_004/INFORMATION" target="_blank">link</a> | EO:MO:DAT:MEDSEA_MULTIYEAR_PHY_006_004/med-cmcc-tem-rean-d | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3AMEDSEA_MULTIYEAR_PHY_006_004" target="_blank">link</a> |

### **Learning outcomes**

In this notebook the SST has been analysed in the Adriatic Sea from 01/01/1987 to 31/12/2019 therefore you will find; 

* How to analyse the SST Time Series in the Adriatic Sea from CMEMS Mediterranean Sea Physics Reanalysis.
* How to generate plots for daily, monthly and annual trends in average temperature.
* How to visualize maps of the average seasonals and anomalies.



<div class="alert alert-info" role="alert">

    
## <a id='TOC_TOP'></a> Contents


 1. [Section 1](#section1): Data Preparation: CMEMS Data.
 
**The CODE SECTION includes:**


 2. [Section 2](#section2): Functions for the Data Aggregation process: Annual, Winter and Summer Seasons Aggregations.
 3. [Section 3](#section3): The SST Data Analysis and Plots: Daily, Monthly, Annual Trends and Standard Deviation.
 4. [Section 4](#section4): The Seasonal and Annual Mean Map Visualizations.
 5. [Section 5](#section5): The Seasonal Anomalies Map Visualizations.
 6. [Section 6](#section6): References.
 7. [Section 7](#section7): Challenge.

<hr>


### Outline

The first section of the Jupyter Notebook describes how to get the data from CMEMS, then it provides the Python code for:
   - Data Aggregation
   - Time Series Analysis
   - Data Visualization

**The Code Section is based on the following elements**

1. Required Python Modules
        
   **The following libraries need to be installed:**
        
            conda install -c conda-forge xarray dask netCDF4 -y
            conda install -c conda-forge cartopy
            pip install statsmodels
            pip install seaborn
            pip install regionmask
            pip install pygeos
            

2. "functions_cmems.py" called in the main programme covers:
    
   The file contains the calling functions to provide necessary methods for the data elaboration. Therefore, in the initial part, it contains the specific functions for the data aggregation, then continuing with time series analysis and visualization functions and finally providing routines for the calculation and visualization of seasonal anomalies. The function names are described in details in the code section step by step.    


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a> [Section 1](#section1): Data Preparation

[Back to top](#TOC_TOP)
    

The first part of the Notebook describes how to get the data from CMEMS. The name of the CMEMS DATA is **Mediterranean Sea Physics Reanalysis**. The dataset name is **med-cmcc-tem-rean-d** and contain 33 years daily information. Thereafter, the dataset is reseized for the Adriatic Sea: 

     Time= 1987-2019
     
     Longitude: 12 to 22 E°  and  Latitude: 37 to 46 N° 

Hence, the Dataset Dimension is:
    
     Time: 12053 
     lat: 216 
     lon: 241
     depth: 1.01823
  
And finally a unique output file "CMEMS_SST" in netCDF format is prepared for the data aggregation process and can easily downloaded from the <a href="https://drive.google.com/drive/u/0/folders/1xTPbAGIs5pmdLXPWyCLLI1ogw3fV6add" target="_blank">following link</a>. The file size is 2.34 GB.
The spatial data in csv format is also present in the <a href="https://drive.google.com/drive/u/0/folders/1xTPbAGIs5pmdLXPWyCLLI1ogw3fV6add" target="_blank">following link</a>.

<div class="alert alert-block alert-warning">

### **Note that**: 
The winter Season is defined from January to April, and the Summer Season is from July to October in the Adriatic Sea,<a href="https://journals.ametsoc.org/view/journals/phoc/27/8/1520-0485_1997_027_1492_tasgcp_2.0.co_2.xml" target="_blank"> REFERENCE LINK</a>.

<div class="alert alert-danger" role="alert">

## <a id='TOC_TOP'></a> CODE SECTION


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a> [Section 2](#section2): Functions for the Data Aggregation process.

[Back to top](#TOC_TOP)

#### **Importing libraries and functions**

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import regionmask
import matplotlib.colors

import functions_cmems as fc

import warnings
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=FutureWarning)

### **Loading the Data**

In [ ]:
ncRawDataFileName = "DATA_PATH/WEkEO_SST_DATA/CMEMS_SST.nc"
fc.areaPerimeter =pd.read_csv ("DATA_PATH/areaAdriatic.csv")
dataOutput = "DATA_PATH/CMEMS_SST_clipped.nc"

"areaPerimeter" is delimiting function over the area of interest. In the spatial data, the 1st column is longitude, the 2nd column is latitude. The Clipped file is saved as "CMEMS_SST_clipped.nc". 

In [ ]:
rawData = xr.open_dataset(ncRawDataFileName)
clippedData = fc.ClipDataOnRegion(rawData, fc.areaPerimeter,dataOutput)

In [ ]:
ncRawDataFileName_annual_mean = "DATA_PATH/CMEMS_SST_clipped.nc"
annualMapsNcFile = "DATA_PATH/CMEMS_SST_clipped_Annual_Mean.nc"

In [ ]:
rawData_annual_mean = xr.open_dataset(ncRawDataFileName_annual_mean)


The clipped file has been indexed in "months" through the XARRAY library ,then averaged by year and finally saved as "CMEMS_SST_clipped_Annual_Mean.nc" by using the <font color='red'> **GenerateAnnualMeanMaps** </font> function

* am1 = t.sel(time=AM(t['time.month']))
* am2 = am1.groupby('time.year').mean('time')

In [ ]:
clippedData_annual_mean = fc.GenerateAnnualMeanMaps(rawData_annual_mean, annualMapsNcFile)

### **The Mean Seasonal Maps have been generated with dimensions (TIME, LAT and LON) from "CMEMS_SST_clipped.nc" file.**
The new aggregated files have been saved as:

##### 1.  "CMEMS_SST_WINTER_SEASON.nc" and is defined between January and April by using <font color='red'>       **"GenerateSeasonalWinter"** </font> function: 

        def WINTER(month):
    
         return (month >= 1) & (month <= 4)
      
         seasonal_data_winter = t.sel(time=WINTER(t['time.month']))
        
        
##### 2. "CMEMS_SST_SUMMER_SEASON.nc" through <font color='red'> **"GenerateSeasonalSummer"** </font> function between July and October:

        def SUMMER(month):
        
         return (month >= 7) & (month <= 10)
      
         seasonal_data_summer = t.sel(time=SUMMER(t['time.month']))

##### **The Seasonal means have been calculated from each saved file:**
         
       seasonal_data_winter1 = seasonal_data_winter.groupby('time.year').mean()
       seasonal_data_summer1 = seasonal_data_summer.groupby('time.year').mean()

##### **The maximum and minimum Temperatures by Season are also printed:**
       
       print("",seasonal_data_winter.thetao.min())
       print("",seasonal_data_winter.thetao.max())
       print("",seasonal_data_summer.thetao.min())
       print("",seasonal_data_summer.thetao.max())

In [ ]:
winter_output= "DATA_PATH/CMEMS_SST_WINTER_SEASON.nc"
summer_output="DATA_PATH/CMEMS_SST_SUMMER_SEASON.nc"

In [ ]:
SeasonWinter = fc.GenerateSeasonalWinter(rawData_annual_mean,winter_output)

In [ ]:
SeasonSummer = fc.GenerateSeasonalSummer(rawData_annual_mean,summer_output)

### **The following 1D outputs are aggregated SST over the Adriatic Sea.**

In [ ]:
NcFile1Doutput = "DATA_PATH/CMEMS_SST_clipped_1D_FIXED_DIM.nc"

In [ ]:
clippedfix1=fc.Generate1DFixDim(rawData_annual_mean,NcFile1Doutput)

In [ ]:
NcFile1DoutputCSV= "DATA_PATH/CMEMS_SST_clipped_1D_FIXED_DIM.csv"

In [ ]:
clippedData1Dcsv = fc.Generate1DFixDimCSV(NcFile1Doutput,NcFile1DoutputCSV)

In [ ]:
clippedData1Dcsv

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a> [Section 3](#section3): The SST Data Analysis and Plots.

[Back to top](#TOC_TOP)

The following plots will visualize:
<ol>
<li>
SST Time Series, Daily Trend in the Adriatic Sea,</li>

<li> 
SST Standard Deviation in the Adriatic Sea,</li>

<li> 
SST Annual Trend in the Adriatic Sea,</li>

<li> 
Monthly Mean SST in the Adriatic Sea.</li>

</ol>


The Time Series analysis have been generated from the previously clipped "CMEMS_SST_clipped_1D_FIXED_DIM" file in CSV format. Therefore,
The <font color='red'> **"faGenerateDailyTimeSeries"** </font> function reads the Daily Mean file through pandas and parse dates with taking the list of "DATE" column. 

In [ ]:
NcFile1DoutputCSV= "DATA_PATH/CMEMS_SST_clipped_1D_FIXED_DIM.csv"

In [ ]:
ts=fc.GenerateDailyTimeSeries(NcFile1DoutputCSV)

The <font color='red'> **"GenerateDailyTimeSeriesSTD"** </font> function starts with the **groupby** method to provide information on data in the "DATE" column in "Monthly Mean".

    fy_dt = file2.groupby(pd.Grouper(freq='M')).mean()

The window size equal to 12 has been choosen for the moving average calculation to calculate standard deviation by year.

     daily_sdT = fy_dt.rolling(window = 12).std()

In [ ]:
ts1=fc.GenerateDailyTimeSeriesSTD(NcFile1DoutputCSV)

The <font color='red'> **"Generate1DTendency"** </font> function shows the Annual Trend in the Adriatic Sea. The input file is a 1-Dimensional file in netCDF format.
Hence, the Linear Regression has been calculated, once the data frame for the Annual Mean has been created.  

    fy_1D= t.mean(dim=(lat_name, lon_name), skipna=True)    
    fy_dt = fy_1D.groupby('time.year').mean()
    df = fy_dt.to_dataframe().reset_index().set_index('year')
    
The horizontal axis has "df.index" by year while vertical axis has the Temperature with the variable name "thetao" in the Cartesian coordinate system.

In [ ]:
ncRawDataFileName_clipped = "DATA_PATH/CMEMS_SST_clipped.nc"
rawData1_clipped = xr.open_dataset(ncRawDataFileName_clipped)

In [ ]:
NcFile1Doutput = "DATA_PATH/CMEMS_SST_clipped_1D_FIXED_DIM.nc"
clippedfix1Tendency=fc.Generate1DTendency(rawData1_clipped,NcFile1Doutput)

The <font color='red'> **"GenerateDailyTimeSeriesPLOT"** </font> function shows the Monthly Mean with Violin Plot in the Adriatic Sea. The input file is a 1-Dimensional file in CSV format. The Data distribution is shown by using the Seaborn Library. 
The month names are converted in their full names with: 

     file2['month'] = [d.strftime('%b') for d in file2.DATE]
     

In [ ]:
ts_monthly=fc.GenerateDailyTimeSeriesPLOT(NcFile1DoutputCSV)

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a> [Section 4](#section4): The Seasonal and Annual Mean Map Visualizations
    
[Back to top](#TOC_TOP)

Each map has been displayed with the following steps:
    
1. Loading the data variable for each selected period of time both seasonal and annual mean with dimensions of (time, lat, lon) and the spatial data to mask out the missing values.
2. The Mean Temperatures for each time coverage have been calculated.
3. Data masking for the area of interest through the vectorized library pygoes.
4. The Plate Carrée projection with the coastline has been selected for the map.
5. A heatmap generation for maximum and minimum temperature toghether with its contour line. 
6. The colour bar has been set.

In [ ]:
t_summer = xr.open_dataset('DATA_PATH/CMEMS_SST_SUMMER_SEASON.nc')
t_winter = xr.open_dataset('DATA_PATH/CMEMS_SST_WINTER_SEASON.nc')
t_annual_mean = xr.open_dataset('DATA_PATH/CMEMS_SST_clipped_Annual_Mean.nc')

In [ ]:
t_summer,t_winter,t_annual_mean

In [ ]:
file_csv_area_1= pd.read_csv('DATA_PATH/areaAdriatic.csv')

##### **The xarray.DataArray with Summer Period and geographical coordinates is shown:**

In [ ]:
temp_summer = t_summer['thetao'][:,:,:]
temp_summer

##### **The xarray.DataArray with Winter Period and geographical coordinates is shown:**

In [ ]:
temp_winter = t_winter['thetao'][:,:,:]
temp_winter

##### **And finally, the xarray.DataArray with Annual Mean and geographical coordinates is as follows: (the Time period is 33 years as noted before).**

In [ ]:
temp_annual_mean = t_annual_mean['thetao'][:,:,:]
temp_annual_mean

**Maximum and Minimum Mean Temperatures for each time coverage follow:**

In [ ]:
temp_summer_av= np.mean(temp_summer[:],axis = 0)
temp_summer_av.min(),temp_summer_av.max()

In [ ]:
temp_winter_av= np.mean(temp_winter[:],axis = 0)
temp_winter_av.min(),temp_winter_av.max()

In [ ]:
temp_annual_mean_av= np.mean(temp_annual_mean[:],axis = 0)
temp_annual_mean_av.min(),temp_annual_mean_av.max()

In [ ]:
lon_name_summer   = temp_summer.lon[:]
lat_name_summer   = temp_summer.lat[:]

lon_name_winter   = temp_winter.lon[:]
lat_name_winter   = temp_winter.lat[:]

lon_name_annual_mean   = temp_annual_mean.lon[:]
lat_name_annual_mean   = temp_annual_mean.lat[:]

**Masking the area for the Adriatic Sea during the Summer Season**

In [ ]:
outline_adriatic = np.array(file_csv_area_1)

region_area_adriatic = regionmask.Regions([outline_adriatic])

In [ ]:
mask_pygeos_area_summer = region_area_adriatic.mask(t_summer.thetao, method="pygeos")
LON, LAT = np.meshgrid(lon_name_summer, lat_name_summer)

In [ ]:
thetao_area_summer = temp_summer_av.values
thetao_area_summer[np.isnan(mask_pygeos_area_summer)] = np.nan

In [ ]:
thetao_area_summer

**Masking the area for the Adriatic Sea during the Winter Season**

In [ ]:
mask_pygeos_area_winter = region_area_adriatic.mask(t_winter.thetao, method="pygeos")
LON1, LAT1 = np.meshgrid(lon_name_winter, lat_name_winter)

In [ ]:
thetao_area_winter = temp_winter_av.values
thetao_area_winter[np.isnan(mask_pygeos_area_winter)] = np.nan

In [ ]:
thetao_area_winter

**Masking the area for the Adriatic Sea for the Annual Mean**

In [ ]:
mask_pygeos_area_annual_mean = region_area_adriatic.mask(t_annual_mean.thetao, method="pygeos")
LON2, LAT2 = np.meshgrid(lon_name_annual_mean, lat_name_annual_mean)

In [ ]:
thetao_area_annual_mean = temp_annual_mean_av.values
thetao_area_annual_mean[np.isnan(mask_pygeos_area_annual_mean)] = np.nan

In [ ]:
thetao_area_annual_mean

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.subplot(projection=ccrs.PlateCarree())

heatmap=temp_summer_av.plot(
    ax=ax,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    cmap="jet",
    shading="auto",
    add_colorbar=False,
    vmin=temp_summer_av.min(),
    vmax=temp_summer_av.max()
)
lines=temp_summer_av.plot.contour(ax=ax,alpha=1,linewidths=0.3,colors = 'k',linestyles='None',levels=60)
# the level of contour lines= (vmax-vmin)*10


g1 = ax.gridlines(draw_labels = True)
g1.xlabel_style = {'size': 16, 'color': 'k'}
g1.ylabel_style = {'size': 16, 'color': 'k'}
#add embellishment


ax.add_feature(cfeature.COASTLINE,linewidths=0.7,alpha=0.9999)



plt.title("CMEMS REANALYSIS: Summer Seasonal Mean SST in the Adriatic Sea\nfrom 1987 to 2019\n",fontweight='bold', size=14)
cbar = plt.colorbar(heatmap)
cbar.ax.set_ylabel('Temperature [°C]',labelpad=+14, rotation=270)

plt.text(17,44,'          Months:\n     July,August\nSeptember,October',fontsize=12,bbox = dict(facecolor = 'gray', alpha = 0.5))
plt.tight_layout()
plt.savefig('image_outputs/SummerSeasonalMean.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.subplot(projection=ccrs.PlateCarree())
heatmap=temp_winter_av.plot(
    ax=ax,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    cmap="jet",
    shading="auto",
    add_colorbar=False,
    vmin=temp_winter_av.min(),
    vmax=temp_winter_av.max()
)

lines=temp_winter_av.plot.contour(ax=ax,alpha=1,linewidths=0.3,colors = 'k',linestyles='None',levels=70)
# the level of contour lines= (vmax-vmin)*10

g1 = ax.gridlines(draw_labels = True)
g1.xlabel_style = {'size': 16, 'color': 'k'}
g1.ylabel_style = {'size': 16, 'color': 'k'}
#add embellishment




ax.add_feature(cfeature.COASTLINE,linewidths=0.7,alpha=0.9999)



plt.title("CMEMS REANALYSIS: Winter Seasonal Mean SST in the Adriatic Sea\nfrom 1987 to 2019\n\n",fontweight='bold', size=14)
cbar = plt.colorbar(heatmap)
cbar.ax.set_ylabel('Temperature [°C]', labelpad=+14,rotation=270)

plt.text(17.5,44,'          Months:\n     January,Februay\n   September,October',fontsize=12,bbox = dict(facecolor = 'gray', alpha = 0.5))
plt.tight_layout()
plt.savefig('image_outputs/WinterSeasonalMean.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.subplot(projection=ccrs.PlateCarree())
hetmap=temp_annual_mean_av.plot(
    ax=ax,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    cmap="jet",
    shading="auto",
    add_colorbar=False,
    vmin=temp_annual_mean_av.min(),
    vmax=temp_annual_mean_av.max()
)

lines=temp_annual_mean_av.plot.contour(ax=ax,alpha=1,linewidths=0.3,colors = 'k',linestyles='None',levels=50)
# the level of contour lines= (vmax-vmin)*10

g1 = ax.gridlines(draw_labels = True)
g1.xlabel_style = {'size': 16, 'color': 'k'}
g1.ylabel_style = {'size': 16, 'color': 'k'}
#add embellishment




ax.add_feature(cfeature.COASTLINE,linewidths=0.7,alpha=0.9999)

plt.title("CMEMS REANALYSIS: Annual Mean SST in the Adriatic Sea\nfrom 1987 to 2019\n",fontweight='bold', size=14)

cbar = plt.colorbar(heatmap)
cbar.ax.set_ylabel('Temperature [°C]', labelpad=+10, rotation=270)


plt.tight_layout()
plt.savefig('image_outputs/AnnualSeasonalMean.png')

plt.show()

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a> [Section 5](#section5): The Seasonal Anomalies Map Visualizations.
    
[Back to top](#TOC_TOP)

Load previously generated files to calculate the Seasonal Anomalies:
1. "Aggregations/CMEMS_SST_SUMMER_SEASON.nc"
2. "Aggregations/CMEMS_SST_WINTER_SEASON.nc"

And,

3. "WEkEO_PART_1_Aggregations/areaAdriatic.csv" to Mask out the missing values.

The Seasonal Anomalies have been calculated through the  <font color='red'> **"computeSenSlopeMap"** </font> function.
"Sen Slope" is a method for robust linear regression. It computes the slope as the median of all slopes (in our case all seasonal mean) between paired values.


The "xr.apply_ufunc" is a vectorization function for unlabeled arrays on xarray objects and it is used to create seasonal maps.

In [ ]:
ncRawDataFileName_summer = "Aggregations/CMEMS_SST_SUMMER_SEASON.nc"
ncRawDataFileName_winter = "Aggregations/CMEMS_SST_WINTER_SEASON.nc"

In [ ]:
NcFileDoutput_summer_anomal = "Aggregations/SummerAnomalyOutput.nc"
NcFileDoutput_winter_anomal = "Aggregations/WinterAnomalyOutput.nc"

In [ ]:
rawData_SUMMER_anomaly = xr.open_dataset(ncRawDataFileName_summer)
rawData_WINTER_anomaly = xr.open_dataset(ncRawDataFileName_winter)

In [ ]:
summerAnomalyFile=fc.computeSenSlopeMap(ncRawDataFileName_summer,NcFileDoutput_summer_anomal)

In [ ]:
winterAnomalyFile=fc.computeSenSlopeMap(ncRawDataFileName_winter,NcFileDoutput_winter_anomal)

In [ ]:
summer_anomaly_vis = xr.open_dataset("Aggregations/SummerAnomalyOutput.nc")

In [ ]:
lon_name_summer   = summer_anomaly_vis.lon[:]
lat_name_summer  = summer_anomaly_vis.lat[:]
time_name_summer  = 'year'
depth_name_summer = 'depth'
temp_summer = summer_anomaly_vis.thetao[:]

In [ ]:
outline_1 = np.array(file_csv_area_1)

region_area_1 = regionmask.Regions([outline_1])

In [ ]:
mask_pygeos_area_1 = region_area_1.mask(summer_anomaly_vis.thetao, method="pygeos")
LON, LAT = np.meshgrid(lon_name_summer, lat_name_summer)

In [ ]:
thetao_area_1 = summer_anomaly_vis.thetao.values
thetao_area_1[np.isnan(mask_pygeos_area_1)] = np.nan

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.subplot(projection=ccrs.PlateCarree())
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#653700","red","white","blue","#00035b"]) 
#new colorbar generation

heatmap=summer_anomaly_vis.thetao.plot(
    ax=ax,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    cmap=cmap.reversed(),
    shading="auto",
    add_colorbar=False,
    vmin=-0.06,
    vmax=0.06
)

lines=summer_anomaly_vis.thetao.plot.contour(ax=ax,alpha=1,linewidths=0.3,colors = 'k',linestyles='None',levels=30)

g1 = ax.gridlines(draw_labels = True)
g1.xlabel_style = {'size': 16, 'color': 'k'}
g1.ylabel_style = {'size': 16, 'color': 'k'}
#add embellishment




ax.add_feature(cfeature.COASTLINE,linewidths=0.7,alpha=0.9999)


plt.title("CMEMS REANALYSIS:SST Summer Season Anomalies in the Adriatic Sea\nfrom 1987 to 2019\n",fontweight='bold', size=14)

plt.tight_layout()
plt.savefig('image_outputs/SummerSEasonAnnomalies.png')

cbar = plt.colorbar(heatmap)
cbar.ax.set_ylabel('Temperature [°C]',labelpad=+10, rotation=270)

plt.text(18,44,'          Months:\n       July,August\nSeptember,October',fontsize=12,bbox = dict(facecolor = 'gray', alpha = 0.5))

plt.tight_layout()

plt.show()


In [ ]:
winter_anomaly_vis = xr.open_dataset("Aggregations/WinterAnomalyOutput.nc")

In [ ]:
lon_name_winter   = winter_anomaly_vis.lon[:]
lat_name_winter  = winter_anomaly_vis.lat[:]
time_name_winter  = 'year'
depth_name_winter = 'depth'
temp_winter = winter_anomaly_vis.thetao[:]

In [ ]:
outline_1 = np.array(file_csv_area_1)

region_area_1 = regionmask.Regions([outline_1])

In [ ]:
mask_pygeos_area_1 = region_area_1.mask(winter_anomaly_vis.thetao, method="pygeos")
LON, LAT = np.meshgrid(lon_name_winter, lat_name_winter)

In [ ]:
thetao_area_1 = winter_anomaly_vis.thetao.values
thetao_area_1[np.isnan(mask_pygeos_area_1)] = np.nan

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.subplot(projection=ccrs.PlateCarree())
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#653700","red","white","blue","#00035b"]) 
#new colorbar generation

heatmap=winter_anomaly_vis.thetao.plot(
    ax=ax,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    cmap=cmap.reversed(),
    shading="auto",
    add_colorbar=False,
    vmin=-0.06,
    vmax=0.06
)

lines=winter_anomaly_vis.thetao.plot.contour(ax=ax,alpha=1,linewidths=0.3,colors = 'k',linestyles='None',levels=30)

g1 = ax.gridlines(draw_labels = True)
g1.xlabel_style = {'size': 16, 'color': 'k'}
g1.ylabel_style = {'size': 16, 'color': 'k'}
#add embellishment




ax.add_feature(cfeature.COASTLINE,linewidths=0.7,alpha=0.9999)


plt.title("CMEMS REANALYSIS:SST Winter Season Anomalies in the Adriatic Sea\nfrom 1987 to 2019\n",fontweight='bold', size=14)

plt.tight_layout()
plt.savefig('image_outputs/WinterSEasonAnnomalies.png')

cbar = plt.colorbar(heatmap)
cbar.ax.set_ylabel('Temperature [°C]',labelpad=+10, rotation=270)

plt.text(18,44,'          Months:\n   January,February\n       March,April',fontsize=12,bbox = dict(facecolor = 'gray', alpha = 0.5))

plt.tight_layout()

plt.show()

<div class="alert alert-block alert-warning">

### Challenge:
    
<div>

The same script can be used in different area of interest for instance, over the Alboran Sea.
    In the following link <a href="https://drive.google.com/drive/u/0/folders/14-IFWR8Ro58BPxJfO4skp3sNqRxIQ6JU" target="_blank">link</a> you can find the input data and the spatial data. 
    
    
    The dataset name is CMEMS_SST_AlboranSea.nc and contain 25 years daily information. Thereafter, the dataset is reseized for the Alboran sea through the alboranSea.csv file:

         Time= 1995-2019

         Longitude: -6 to -2 E°  and  Latitude: 34 to -37 N° 
    Hence, the Dataset Dimension is:

         Time: 9131 
         lat: 39 
         lon: 97
         depth: 1.01823
      

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a> [Section 6](#section6): References
    
* WEkEO <a href="https://www.wekeo.eu/" target="_blank">LINK</a>

* Copernicus Marine <a href="https://marine.copernicus.eu/" target="_blank">LINK</a>
    
* Mediterranean Sea Physics Reanalysis <a href="https://resources.marine.copernicus.eu/product-detail/MEDSEA_MULTIYEAR_PHY_006_004/INFORMATION" target="_blank">LINK</a>

* WEkEO Data-- EO:MO:DAT:MEDSEA_MULTIYEAR_PHY_006_004 <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3AMEDSEA_MULTIYEAR_PHY_006_004&initial=1" target="_blank">LINK</a>

* Global Climate Observing Sysem - GCOS WMO: Sea Surface Temperature (SST) <a href="https://gcos.wmo.int/en/essential-climate-variables/sst" target="_blank">LINK</a>
    

* "The Adriatic Sea General Circulation. Part I: Air–Sea Interactions and Water Mass Structure"  <a href="https://doi.org/10.1175/1520-0485(1997)027" target="_blank">DOI</a>